In [1]:
import sys
sys.path.append("../src")

In [ ]:
from model import UnetModel
from skimage.io import imshow
%matplotlib inline

Using TensorFlow backend.


In [ ]:
if True:
    losses = []
    for size in [(512, 512), (128, 128), (256, 256)]:
        for batch_size in [4, 32, 16]:
            for fold_num in range(1, 2):
                batches = 32 * 100 // batch_size
                model = UnetModel(f"unet__{size}_{batch_size}_{fold_num}", input_size=size, patience=10)
                model.unet.load_weights("./unet__(224, 224)_8_1.best.weights.h5py")
                print(f"Size = {size}, batch size = {batch_size}")
                learn_rates = [0.0001, 0.00001, 0.000001]
                epochs_list = [50, 50, 50]

                for lr, epochs in zip(learn_rates, epochs_list):
                    train_loss, val_loss = model.fit_resized(lr, epochs, fold_num, batch_size=batch_size, batches=batches)
                losses.append((train_loss, val_loss))

../src/unet.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`
  model = Model(input=inputs, outputs=conv12)


Size = (512, 512), batch size = 4


../src/model.py:53: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=800, validation_data=<generator..., validation_steps=800, callbacks=[<callback..., epochs=50)`
  callbacks=self.callbacks)


Epoch 1/50
800/800 [==============================] - 979s - loss: 0.0410 - dice_coef: 0.9153 - val_loss: 0.1345 - val_dice_coef: 0.8154
Epoch 2/50
760/800 [===========================>..] - ETA: 37s - loss: 0.0356 - dice_coef: 0.9237

In [ ]:
img_size = (512, 512)
model = UnetModel("prediction", input_size=img_size)
model.unet.load_weights(f"./unet__{img_size}_4_1.best.weights.h5py")
from generator import train_generator_resized, valid_generator_resized
batch = next(valid_generator_resized(n_fold=1, size=img_size, batch_size=2))

In [ ]:
imshow(model.unet.predict(batch[0])[0].squeeze() * 255)

In [ ]:
imshow(batch[1][0].squeeze().astype('uint8'))

In [ ]:
from predictor import Predictor
predictor = Predictor(model.unet)
predictor.create_resized_submission(img_size)